# Ironbox: MCP Integration

This notebook demonstrates how Ironbox integrates with the Model Context Protocol (MCP) to extend its capabilities with external services and tools.

## Overview

The Model Context Protocol (MCP) is a standardized way for AI systems to interact with external tools and resources. Ironbox integrates MCP through several components:

1. **MCP Client**: Connects to MCP servers and manages communication
2. **MCP Agent**: Specialized agent for MCP operations
3. **Tool Repository**: Registers MCP tools alongside local tools
4. **MCP Servers**: External servers that provide tools and resources

Let's explore how these components work together to extend Ironbox's capabilities.

## Setup

First, let's import the necessary modules and initialize the MCP components.

In [ ]:
import os
import sys
import json
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Import Ironbox components
from ironbox.mcp.client import MCPClient
from ironbox.agents.mcp_agent import MCPAgent
from ironbox.core.agent_core import AgentCore

# Initialize the MCP client
mcp_client = MCPClient()

# Initialize the MCP agent
mcp_agent = MCPAgent(mcp_client)

# Initialize the Agent Core
agent_core = AgentCore()
agent_core.register_agent("mcp", mcp_agent.process_query)

print("MCP components initialized successfully.")

## MCP Servers

Let's explore the MCP servers that are connected to Ironbox.

In [ ]:
# Function to list MCP servers
def list_mcp_servers():
    print("=== Connected MCP Servers ===")
    servers = mcp_client.list_servers()
    
    if not servers:
        print("No MCP servers are currently connected.")
        return []
    
    for i, server in enumerate(servers):
        print(f"Server {i+1}: {server['name']}")
        print(f"  Endpoint: {server['endpoint']}")
        print(f"  Status: {server['status']}")
        print(f"  Last Connected: {server['last_connected']}")
        print(f"  Capabilities: {len(server['capabilities'])} tools and resources")
        print()
    
    return servers

# List connected MCP servers
servers = list_mcp_servers()

## Weather MCP Server

Let's start by examining the Weather MCP Server, which provides weather information through the OpenWeather API.

In [ ]:
# Function to get server details
def get_server_details(server_name):
    print(f"=== {server_name} Server Details ===")
    server = mcp_client.get_server(server_name)
    
    if not server:
        print(f"Server '{server_name}' not found.")
        return None
    
    print(f"Name: {server['name']}")
    print(f"Endpoint: {server['endpoint']}")
    print(f"Status: {server['status']}")
    print(f"Last Connected: {server['last_connected']}")
    
    print("\nTools:")
    for tool in server['capabilities'].get('tools', []):
        print(f"  - {tool['name']}: {tool['description']}")
    
    print("\nResources:")
    for resource in server['capabilities'].get('resources', []):
        print(f"  - {resource['uri']}: {resource.get('description', 'No description')}")
    
    print("\nResource Templates:")
    for template in server['capabilities'].get('resource_templates', []):
        print(f"  - {template['uriTemplate']}: {template.get('description', 'No description')}")
    
    return server

# Get details for the Weather MCP Server
weather_server = get_server_details("weather")

## Using MCP Tools

Now let's use the tools provided by the Weather MCP Server.

In [ ]:
# Function to execute an MCP tool
def execute_mcp_tool(server_name, tool_name, arguments):
    print(f"=== Executing MCP Tool: {tool_name} ===")
    print(f"Server: {server_name}")
    print(f"Arguments: {json.dumps(arguments, indent=2)}")
    
    result = mcp_client.execute_tool(server_name, tool_name, arguments)
    
    print("\nResult:")
    print(json.dumps(result, indent=2))
    print()
    
    return result

# Execute the get_forecast tool
forecast = execute_mcp_tool("weather", "get_forecast", {"city": "San Francisco", "days": 3})

## Accessing MCP Resources

MCP servers can also provide resources, which are static or dynamic data sources. Let's access some resources from the Weather MCP Server.

In [ ]:
# Function to access an MCP resource
def access_mcp_resource(server_name, uri):
    print(f"=== Accessing MCP Resource: {uri} ===")
    print(f"Server: {server_name}")
    
    resource = mcp_client.access_resource(server_name, uri)
    
    print("\nResource Content:")
    print(resource)
    print()
    
    return resource

# Access the current weather resource for San Francisco
sf_weather = access_mcp_resource("weather", "weather://San Francisco/current")

# Access the current weather resource for New York
ny_weather = access_mcp_resource("weather", "weather://New York/current")

## MCP Agent

The MCP Agent provides a natural language interface to MCP operations. Let's see how it works.

In [ ]:
# Function to use the MCP Agent
def use_mcp_agent(query):
    print(f"=== MCP Agent Query: '{query}' ===")
    response = mcp_agent.process_query(query)
    print(f"Response: {response}\n")
    return response

# Use the MCP Agent for various queries
queries = [
    "What's the weather like in Tokyo?",
    "Get me a 5-day forecast for London",
    "List all available MCP servers",
    "What tools does the weather server provide?"
]

for query in queries:
    use_mcp_agent(query)

## Creating a Custom MCP Server

Let's examine how to create a custom MCP server to extend Ironbox's capabilities.

In [ ]:
# Import the necessary modules for creating an MCP server
from ironbox.mcp.weather_server import WeatherServer

# Function to demonstrate creating a custom MCP server
def demonstrate_custom_mcp_server():
    print("=== Creating a Custom MCP Server ===")
    print("This is a simplified example of how to create a custom MCP server.")
    print("In a real implementation, you would create a new Python file with your server code.\n")
    
    # Show the basic structure of an MCP server
    server_code = """
#!/usr/bin/env node
import { Server } from '@modelcontextprotocol/sdk/server/index.js';
import { StdioServerTransport } from '@modelcontextprotocol/sdk/server/stdio.js';
import {
  CallToolRequestSchema,
  ErrorCode,
  ListResourcesRequestSchema,
  ListResourceTemplatesRequestSchema,
  ListToolsRequestSchema,
  McpError,
  ReadResourceRequestSchema,
} from '@modelcontextprotocol/sdk/types.js';
import axios from 'axios';

class CustomMCPServer {
  private server: Server;

  constructor() {
    this.server = new Server(
      {
        name: 'custom-mcp-server',
        version: '0.1.0',
      },
      {
        capabilities: {
          resources: {},
          tools: {},
        },
      }
    );

    this.setupToolHandlers();
    this.setupResourceHandlers();
    
    // Error handling
    this.server.onerror = (error) => console.error('[MCP Error]', error);
  }

  private setupToolHandlers() {
    this.server.setRequestHandler(ListToolsRequestSchema, async () => ({
      tools: [
        {
          name: 'custom_tool',
          description: 'A custom tool that does something useful',
          inputSchema: {
            type: 'object',
            properties: {
              param1: {
                type: 'string',
                description: 'First parameter',
              },
              param2: {
                type: 'number',
                description: 'Second parameter',
              },
            },
            required: ['param1'],
          },
        },
      ],
    }));

    this.server.setRequestHandler(CallToolRequestSchema, async (request) => {
      if (request.params.name !== 'custom_tool') {
        throw new McpError(
          ErrorCode.MethodNotFound,
          `Unknown tool: ${request.params.name}`
        );
      }

      // Tool implementation goes here
      return {
        content: [
          {
            type: 'text',
            text: 'Custom tool result',
          },
        ],
      };
    });
  }

  private setupResourceHandlers() {
    this.server.setRequestHandler(ListResourcesRequestSchema, async () => ({
      resources: [
        {
          uri: 'custom://resource',
          name: 'Custom Resource',
          mimeType: 'application/json',
          description: 'A custom resource',
        },
      ],
    }));

    this.server.setRequestHandler(ReadResourceRequestSchema, async (request) => {
      if (request.params.uri !== 'custom://resource') {
        throw new McpError(
          ErrorCode.InvalidRequest,
          `Invalid URI: ${request.params.uri}`
        );
      }

      return {
        contents: [
          {
            uri: request.params.uri,
            mimeType: 'application/json',
            text: JSON.stringify({ data: 'Custom resource data' }, null, 2),
          },
        ],
      };
    });
  }

  async run() {
    const transport = new StdioServerTransport();
    await this.server.connect(transport);
    console.error('Custom MCP server running on stdio');
  }
}

const server = new CustomMCPServer();
server.run().catch(console.error);
    """
    
    print("Basic structure of an MCP server:")
    print(server_code)
    
    # Show how to register the server in the MCP settings
    mcp_settings = """
{
  "mcpServers": {
    "weather": {
      "command": "node",
      "args": ["/path/to/weather-server/build/index.js"],
      "env": {
        "OPENWEATHER_API_KEY": "your-api-key"
      }
    },
    "custom": {
      "command": "node",
      "args": ["/path/to/custom-server/build/index.js"],
      "env": {
        "CUSTOM_API_KEY": "your-custom-api-key"
      }
    }
  }
}
    """
    
    print("\nMCP settings configuration:")
    print(mcp_settings)
    
    # Show how to use the custom server
    print("\nUsing the custom server:")
    print("mcp_client.execute_tool('custom', 'custom_tool', {'param1': 'value1', 'param2': 42})")
    print("mcp_client.access_resource('custom', 'custom://resource')")

# Demonstrate creating a custom MCP server
demonstrate_custom_mcp_server()

## Integrating MCP Tools with Agent Frameworks

Let's see how MCP tools are integrated with the React and Plan agent frameworks.

In [ ]:
# Function to demonstrate MCP tool integration with agent frameworks
def demonstrate_mcp_tool_integration():
    print("=== MCP Tool Integration with Agent Frameworks ===")
    
    # Get the React Framework
    react_framework = agent_core.frameworks.get("react")
    
    # List all tools available to the React Framework
    print("Tools available to the React Framework:")
    for tool_name in agent_core.tools.keys():
        if tool_name.startswith("mcp_"):
            print(f"  - {tool_name} (MCP Tool)")
        else:
            print(f"  - {tool_name} (Local Tool)")
    
    # Process a query that might use an MCP tool
    query = "What's the weather like in Paris right now?"
    print(f"\nProcessing query with React Framework: '{query}'")
    
    from ironbox.core.agent_core import AgentState
    state = AgentState(query=query, session_id="demo-session")
    result_state = react_framework.process(state)
    
    print(f"Response: {result_state.response}")
    
    # Show the thinking steps
    print("\nThinking steps:")
    for i, step in enumerate(result_state.metadata.get('steps', [])):
        print(f"Step {i+1}:")
        print(f"  Thought: {step.thought}")
        print(f"  Action: {step.action}")
        print(f"  Action Input: {step.action_input}")
        print(f"  Observation: {step.observation}\n")

# Demonstrate MCP tool integration
demonstrate_mcp_tool_integration()

## MCP Tool Registration

Let's examine how MCP tools are registered with the Tool Repository.

In [ ]:
# Function to demonstrate MCP tool registration
def demonstrate_mcp_tool_registration():
    print("=== MCP Tool Registration ===")
    
    # Get the list of MCP servers
    servers = mcp_client.list_servers()
    
    if not servers:
        print("No MCP servers are currently connected.")
        return
    
    # For each server, show how its tools are registered
    for server in servers:
        print(f"\nServer: {server['name']}")
        tools = server['capabilities'].get('tools', [])
        
        if not tools:
            print("  No tools provided by this server.")
            continue
        
        print("  Tools:")
        for tool in tools:
            tool_name = f"mcp_{server['name']}_{tool['name']}"
            print(f"    - {tool_name}")
            print(f"      Description: {tool['description']}")
            print(f"      Input Schema: {json.dumps(tool.get('inputSchema', {}), indent=6)}")
            
            # Show the wrapper function
            wrapper_code = f"""
def {tool_name}(**kwargs):
    """Wrapper for the {tool['name']} tool from the {server['name']} MCP server."""
    return mcp_client.execute_tool("{server['name']}", "{tool['name']}", kwargs)
            """
            print(f"      Wrapper Function: {wrapper_code}")
    
    # Show how tools are registered with the Agent Core
    register_code = """
# Register MCP tools with the Agent Core
for server_name, server in mcp_client.servers.items():
    for tool in server['capabilities'].get('tools', []):
        tool_name = f"mcp_{server_name}_{tool['name']}"
        wrapper_function = create_tool_wrapper(server_name, tool['name'])
        agent_core.register_tool(tool_name, wrapper_function)
    """
    
    print("\nTool Registration Code:")
    print(register_code)

# Demonstrate MCP tool registration
demonstrate_mcp_tool_registration()

## Conclusion

In this notebook, we've explored how Ironbox integrates with the Model Context Protocol (MCP):

1. **MCP Servers**: External servers that provide tools and resources
2. **MCP Client**: Connects to MCP servers and manages communication
3. **MCP Tools**: External tools that can be used by agent frameworks
4. **MCP Resources**: Static or dynamic data sources provided by MCP servers
5. **MCP Agent**: Specialized agent for MCP operations
6. **Tool Registration**: How MCP tools are registered with the Tool Repository
7. **Custom MCP Servers**: How to create custom MCP servers to extend Ironbox's capabilities

MCP integration allows Ironbox to extend its capabilities with external services and tools, making it a more powerful and flexible system for managing Kubernetes clusters and other tasks.